#### Requires installation of wkhtmltox 

In [32]:
import pandas as pd
import numpy as np
import PySimpleGUI as sg
import lime
import lime.lime_tabular as lime_tabular
import io
import imgkit
path_wkhtmltoimg = r'C:\Program Files\wkhtmltopdf\bin\wkhtmltoimage.exe'
config = imgkit.config(wkhtmltoimage = path_wkhtmltoimg)
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

In [33]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.24.1.


In [34]:
import joblib
clf = joblib.load(r'NewRF.mod')

In [35]:
# import raw training set
DFtrain = pd.read_csv('https://raw.githubusercontent.com/KushajveerSingh/ds_cup/main/data_orig/train.csv')

In [36]:
colNames = ['avg_card_debt',
'card_age',
'non_mtg_acc_past_due_12_months_num',
'inq_12_month_num',
'uti_card'
]

In [37]:
DFtrainX = DFtrain[colNames]
DFtrainY = DFtrain.iloc[:,-1]

In [38]:
TestInstance = pd.DataFrame(columns=colNames)

In [39]:
def askPopUp():
    return True if sg.PopupYesNo('Do you wish to add this for future learning?') == 'Yes' else False

In [40]:
def addValueToCSV(arr):
    nextline = ""
    for x in arr:
        nextline += str(x) + ","
    nextline = nextline[:-1]
    with open('newvalues.csv','a') as f:
        f.write(nextline)
        f.write("\n")
        f.flush()

In [41]:
sg.theme('LightGrey3')   # Add a touch of color
layout = []
for col in colNames:
    layout.append([sg.Text(col,size=(34,1)), sg.InputText()])
layout.append([sg.Image(key='-IMAGE2-'),sg.Text("Fill in details to check Risk",key='-ISRISK-')])
layout.append([sg.Button('Ok'), sg.Button('Incorrect') ,sg.Button('Cancel')])

window = sg.Window('Enter Details', layout)# Create the Window
vals = {}
try:
    while True:# Event Loop to process "events" and get the "values" of the inputs
        event, values = window.read()
        if event == sg.WIN_CLOSED or event == 'Cancel': # if user closes window or clicks cancel
            window.close()
            break
        elif event == 'Ok':
            for val,col in zip(values,colNames):
                vals[col] = float(values[val]) #Saving as backup
            test_instance = np.array([x for x in vals.values()])
            trgtNames = [x for x in set(DFtrainY)]
            explainer = lime_tabular.LimeTabularExplainer(DFtrainX,"classification",feature_names=colNames,class_names=trgtNames,discretize_continuous=False)
            exp = explainer.explain_instance(test_instance,clf.predict_proba,num_features=5,num_samples=5000,model_regressor=None)
            htmlExp = exp.as_html()
            imgkit.from_string(htmlExp, '_out.jpg',config=config)
            
            img = Image.open('_out.jpg', mode='r')
            width, height = img.size
            # Setting the points for cropped image
            left = 0
            top = 0
            right = width*2/3
            bottom = height-30
            
            img = img.crop((left, top, right, bottom)) # Cropped image of above dimension
            img_byte_arr = io.BytesIO()
            img.save(img_byte_arr, format='PNG')
            img_byte_arr = img_byte_arr.getvalue()
            window['-IMAGE2-'].update(data=img_byte_arr)
            exp.predict_proba[-1]
            txt = ""
            eval_prob = False
            if(exp.predict_proba[-1]>0.5758):
                txt = "Risky"
                eval_prob = True
            else:
                txt = "Not Risky"
            window["-ISRISK-"].update(txt)
            if(askPopUp()):
                saveVals = list(test_instance)
                saveVals.extend([eval_prob,eval_prob])
                addValueToCSV(saveVals)
        elif event == 'Incorrect':
             if(askPopUp()):
                 saveVals = list(test_instance)
                 saveVals.extend([eval_prob,not eval_prob])
                 addValueToCSV(saveVals)
except:
    print("Error, Cancelled")
    window.close()
print()

Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               



In [42]:
window.close() #Use this as a backup way to kill the process.